In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

![](https://docs.dask.org/en/latest/_images/dask_icon.svg)

# Introduction to Dask

- If data is more than RAM size, certainly a Data Scientist will look for another tool and first tool which might come in sight is Dask. In Dask the Dask DataFrame is no doubtly going to be a huge support to analyse huge amount of data. A Dask DataFrame, is made of many small Pandas DataFrame.
- Can run in parallel system like cluster or on single machine.
- If Data is larger than RAM. No Problem some part will be in memory some part on disc but computation go on single machine. 

# Installation 

- On Kaggle machines Dask is allready installed. 
- But if you want to install Dask on any machine then it can be installed using conda and pip both
- Installing Dask using conda using following line of code

conda install dask

- If you have installed Anaconda then Dask will be installed with it. Further if required it can be upgraded. 

- Dask can be installed using pip too.

pip install dask  : But it will only install core Dask Part

- In order to install complete Dask following code can be used

pip install "dask[complete]"
conda install "dask[complete]"

# About DataSet used for this kernel

Dataset i have used from kaggle itself. This Dataset about importing stuffs to India and Exporting Stuffs from India. Following Information i have just copy pasted from Data Official site on kaggle 

---------------------------------------------------

- Context
India is one of the fastest developing nations of the world and trade between nations is the major component of any developing nation. This dataset includes the trade data for India for commodities in the HS2 basket.

For more, visit GitHub

- Content
The dataset consists of trade values for export and import of commodities in million US$. The dataset is tidy and each row consists of a single observation.

- Acknowledgements
The data is scraped using Selenium Webdriver from the Department of Commerce, Government of India.

Data set has been collect and uploaded to Kaggle by  Lakshya Agarwal  
https://www.kaggle.com/lakshyaag



# More about Dataset 

taken from kernel 
https://www.kaggle.com/shubhamsinghgharsele/analysis-on-indian-import-export


In both the File we have 5 columns each.

HSCode - HS stands for Harmonized System. It was developed by the WCO (World Customs Organization) as a multipurpose international product nomenclature that describes the type of good that is shipped HS Code Structure

The HS code can be described as follows:

It is a six-digit identification code. It has 5000 commodity groups. Those groups have 99 chapters. Those chapters have 21 sections. It’s arranged in a legal and logical structure. Well-defined rules support it to realize uniform classification worldwide.

the HSCode in column is 99 chapters

Reference HSCode List

Commodity - the column contain chapter wise commodity category. In each commodity Category there are various commodities.

A commodity is an economic good or service that has full or substantial fungibility: that is, the market treats instances of the good as equivalent or nearly so with no regard to who produced them.

Reference

Value - values for export and import of commodities in million US $.
Country - Country Imported From/ Exported To
Year - Year in which comodities where Imported/Exported which is in between 2010 to 2018.

# Importing Dask 

In [ ]:
 import dask.dataframe as dd

# Reading the Data Files

In [ ]:
exportDf = dd.read_csv('/kaggle/input/india-trade-data/2018-2010_export.csv')

In [ ]:
exportDf.head()

In [ ]:
# Finding information about DataFrame
exportDf.info()

In [ ]:
# Schema of DataFrame
exportDf.dtypes

In [ ]:
importDf = dd.read_csv('/kaggle/input/india-trade-data/2018-2010_import.csv')

In [ ]:
importDf.head()

In [ ]:

set(importDf.HSCode) == set(exportDf.HSCode)

# Function compute perform the computation.

In [ ]:
importDf.value.min().compute()

# Data Filtering

In [ ]:
importDf.describe().compute()

In [ ]:
#Getting number of partition
importDf.npartitions

# Data Filtering 
- Getting all the imports where import value is greater than 66

In [ ]:
filteredVal =importDf[importDf.value > 66]
filteredVal.head()

In [ ]:
filteredVal =importDf[(importDf.value > 66) & (importDf.value < 100)]
filteredVal.head()

# Data Aggregation

- Mean value grouped by  HSCode

In [ ]:
meanVal = importDf.groupby("HSCode").mean()
meanVal.compute()

In [ ]:
# Data joining 
joinedData = importDf.merge(exportDf, on = "HSCode",how="inner")
joinedData.compute().head()

# Comparison of performance between pandas and dask DataFrame

In [ ]:
# Reading a File using Pandas
%timeit pd.read_csv("/kaggle/input/india-trade-data/2018-2010_import.csv")

In [ ]:
%timeit dd.read_csv("/kaggle/input/india-trade-data/2018-2010_import.csv")

## For file reading Dask is much faster than Pandas

### Comparison of code efficiency for Data Filtering

In [ ]:
importDfPd = pd.read_csv('/kaggle/input/india-trade-data/2018-2010_import.csv')
importDfDd = dd.read_csv('/kaggle/input/india-trade-data/2018-2010_import.csv')

In [ ]:
# Filtering using Dask
%timeit val = importDfDd[importDfDd.value >66].compute()

In [ ]:
# Filtering using Pandas
%timeit val = importDfPd[importDfPd.value >66]

### We can observe that time is very large for Dask. Can we decrease it? Let us increase number of Partitions.

In [ ]:
importDfDd.head()

In [ ]:
importDfDd1 = importDfDd.repartition(npartitions = 100)

In [ ]:
importDfDd.npartitions

In [ ]:
# Filtering using Dask
%timeit val = importDfDd1[importDfDd1.value >66].compute()

In [ ]:
importDfDd1 = importDfDd.repartition(npartitions = 1)
%timeit val = importDfDd1[importDfDd1.value >66].compute()